In [1]:
import matplotlib
matplotlib.use('Agg')
import PyVal
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import ipyparallel as ipp
from tqdm import tqdm_notebook
from functools import reduce

In [2]:
%matplotlib
idx = pd.IndexSlice
rc = ipp.Client()#('/home/julian/.ipython/profile_ssh/security/ipcontroller-client.json')
dview = rc.load_balanced_view()
print(rc.ids)

def send_file(dview, local_path, remote_path):
    """Send a single file"""
    print("sending file", local_path, remote_path)
    import io, os
    
    def _recv(remote_path, bdata, mtime):
        import io, os
        if os.path.exists(remote_path):
            os.unlink(remote_path)
        with io.open(remote_path, 'wb') as f:
            f.write(data)
        os.utime(remote_path, (mtime, mtime))
    
    st = os.stat(local_path)
    with io.open(local_path, 'rb') as f:
        data = f.read()
    
    yield dview.apply_async(_recv, remote_path, data, st.st_mtime)
    
#TODO: use chrpath PyVal.so -r . to make libtrng visible
#for el in send_file(dview, '../build/PyVal.cpython-36m-x86_64-linux-gnu.so','./PyVal.so'):
    
    print(el.get())
for el in send_file(dview, '../external/lib/libtrng4.so.19', './libtrng4.so.19'):
    pass
for el in send_file(dview, '../external/lib/libtrng4.so', './libtrng4.so'):
    pass

with rc[:].sync_imports():
    import PyVal
    import numpy as np


Using matplotlib backend: TkAgg
Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json


OSError: Connection file '~/.ipython/profile_default/security/ipcontroller-client.json' not found.
You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.

In [6]:
inf = open('res_0_1.pkl', 'rb')
t1 = pickle.load(inf)
inf.close()
inf = open('res_0_2.pkl', 'rb')
t2 = pickle.load(inf)
inf.close()
inf = open('res_0_3.pkl', 'rb')
t3 = pickle.load(inf)
inf.close()
inf = open('res_0_4.pkl', 'rb')
t4 = pickle.load(inf)
inf.close()

In [ ]:
def flatten_input(cell):
    if type(cell) is list:
        return cell[0]
    else:
        return cell
df1 = pd.DataFrame(t1)
df1 = df1.applymap(flatten_input)
df1.set_index(['N', 'p', 'val'], inplace=True)
df2 = pd.DataFrame(t2)
df2 = df2.applymap(flatten_input)
df2.set_index(['N', 'p', 'val'], inplace=True)
df3 = pd.DataFrame(t3)
df3 = df3.applymap(flatten_input)
df3.set_index(['N', 'p', 'val'], inplace=True)
df4 = pd.DataFrame(t4)
df4 = df4.applymap(flatten_input)
df4.set_index(['N', 'p', 'val'], inplace=True)
combined = pd.concat([df1, df2, df3, df4])

In [ ]:
#@ipp.require('PyVal', 'numpy')
def run_sim(N, val, pList, T, r):
    import numpy as np
    nw = PyVal.BS_Network()
    MList = []
    rsList= []
    assetList= []
    deltaList = []
    solventList = []
    MVarList = []
    rsVarList= []
    assetVarList= []
    deltaVarList = []
    solventVarList = []
    #vOutList = []
    for p in pList:
        print("Runing N=" +str(N)+ ", val="+str(val)+", p="+str(p))
        nw.run(N, p, val, 2, T, r, 200, 800)
        MList.append(nw.get_M())
        rsList.append(np.array(nw.get_rs())[:,0])
        assetList.append(np.array(nw.get_assets())[:,0])
        solventList.append(np.array(nw.get_solvent())[:,0])
        deltaList.append(nw.get_delta_jacobians())
        MVarList.append(nw.get_M_var())
        rsVarList.append(np.array(nw.get_rs_var())[:,0])
        assetVarList.append(np.array(nw.get_assets_var())[:,0])
        solventVarList.append(np.array(nw.get_solvent_var())[:,0])
        deltaVarList.append(nw.get_delta_jacobians_var())
    res = {'N': N, 'conn': (N*np.array(pList)).tolist(), 'T':T, 'r': r ,'p': pList, 'val': val, 'M': MList, 'M var': MVarList,\
           'Assets': assetList, 'Assets var': assetVarList,\
		'RS': rsList, 'Delta': deltaList, 'Solvent': solventList, 'RS var': rsVarList, 'Delta var': deltaVarList, 'Solvent var': solventVarList}
    return res

# callapses a list of results into a single result.
# returns a tuple: (Collapsed result, list of results with wrong parameters)
# TODO: allow for collaps along, r, T, val, N. for now only p
# TODO: sort along collapse axis
def combine_results(*results):
    r0 = results[0]
    res_new = r0.copy()
    failed = []
    if type(r0) is not dict:
        print("WARNING! Did recieve non-dict type as parameter")
    

    def combine(el):
        if r0['val'] != el['val'] or r0['N'] != el['N']\
        or r0['T'] != el['T'] or r0['r'] != el['r']:
            return False
        for key, value in el.items():
            if key == 'val' or key == 'N' or key == 'T' or key == 'r':
                continue
            if type(el[key]) is not list:
                print("WARNING! Key: " + key + " is not a list")
            res_new[key] = res_new[key] + el[key]
        return True
            
    for el in results[1:]:
        if type(el) is not dict:
            print("WARNING! Did recieve non-dict type as parameter")
        if not combine(el):
            failed.append(el)
            
    return (res_new, failed)

def test1(n):
    r = 0
    for i in range(n):
        r += i
    return r

def pList(N, pts):
    import numpy as np
    return np.union1d(np.linspace(0.0,1.8/N,pts), np.linspace(0.7/N,1.3/N,pts))

# Parameters
## Black Scholes
    - T = 1
    - r = 0
    
## Network Parameters
    - N = [15, 25, 50, 75, 100, 150, 200, 300]
    - <k> = n*p = [0 .. 1.8]
    - M => ER(N,p)

In [ ]:
dview.block=False
in_0_1 = [(N,0.1, [el], 1.0, 0.0) for N in [20, 30, 50, 75, 100, 200] for el in pList(N,10)]#, (25,0.1,pList(25,8)), (50,0.1,pList(50,6)), (75, 0.1, pList(75,6))]#, (100, 0.1, pList(100,4)), (200, 0.1, pList(200,2))]
in_0_2 = [(N,0.2, [el], 1.0, 0.0) for N in [20, 30, 50, 75, 100, 200] for el in pList(N,10)]
in_0_3 = [(N,0.3, [el], 1.0, 0.0) for N in [20, 30, 50, 75, 100, 200] for el in pList(N,10)]
in_0_4 = [(N,0.4, [el], 1.0, 0.0) for N in [20, 30, 50, 75, 100, 200] for el in pList(N,10)]
in_0_5 = [(N,0.5, [el], 1.0, 0.0) for N in [20, 30, 50, 75, 100, 200] for el in pList(N,10)]
async_res = []
ids = []
for el in in_0_2:
    b_re = dview.apply_async(run_sim, *el)
    async_res.append(b_re)
    ids.extend(b_re.msg_ids)
rc.get_result(ids, owner=False).wait_interactive()

  64/80 tasks finished after 1693 s

In [4]:
#res_0_5 = [r.get() for r in async_res]
output = open('res_0_2.pkl', 'wb')
pickle.dump(res_0_2, output)
output.close()

NameError: name 'res_0_2' is not defined

In [15]:
N = 100
val = 0.4
test = combined.loc[idx[N,:,val],['Delta','Delta var']]
test_arr = test.values
x = float(N)*test.reset_index(level=[0,2]).index.values
y = [np.sum(el)/float(N) for el in test_arr[:,0]]
plt.plot(x,y)
#lt.axis( [0., 2., 1., 1.5])
plt.show()

In [4]:
ax = plt.subplots(nrows=4, ncols=4, sharex=True, sharey=True)

In [10]:
#res = [r.get() for r in async_res]
results = []
res = res_0_4
tmp = combine_results(*res)
results.append(tmp[0])
while len(tmp[1]):
    tmp = combine_results(*tmp[1])
    results.append(tmp[0])
results[1]

{'Assets': [array([1.01124903, 1.01189336, 1.0108696 , 1.01146834, 1.01155754,
         1.01187135, 1.01081859, 1.01116602, 1.01139831, 1.01145916,
         1.0111476 , 1.01131964, 1.01141692, 1.01195999, 1.01124664,
         1.01105331, 1.01132615, 1.01148874, 1.01114519, 1.01172413,
         1.01152202, 1.01102294, 1.01104297, 1.01124928, 1.01107539,
         1.01133666, 1.01127269, 1.01080839, 1.01121245, 1.01142305]),
  array([1.01124903, 1.01189336, 1.0108696 , 1.01146834, 1.01155754,
         1.01187135, 1.01081859, 1.01116602, 1.01139831, 1.01145916,
         1.0111476 , 1.01131964, 1.01141692, 1.01195999, 1.01124664,
         1.01105331, 1.01132615, 1.01148874, 1.01114519, 1.01172413,
         1.01152202, 1.01102294, 1.01104297, 1.01124928, 1.01107539,
         1.01133666, 1.01127269, 1.01080839, 1.01121245, 1.01142305]),
  array([1.01124903, 1.01189336, 1.0108696 , 1.01146834, 1.01155754,
         1.01187135, 1.01081859, 1.01116602, 1.01139831, 1.01145916,
         1.0111476 ,

In [50]:
bb = combine_results(*aa[1])
bb[1]

[]